In [2]:
import pandas as pd

In [41]:
seccs = []
values = []
curr = []
with open("/home/tyoc213/Escritorio/LOGS/debug_run-503562a578c6-root-20200726T195957Z-001/debug_run-503562a578c6-root/metrics.txt", "r") as f:
    ls = f.read().split("\n")
    for x in ls:
        if len(x) > 1:
            parts = x.split("\t")
            if len(parts) == 1:
                seccs.append(parts[0].replace("[", "").replace("]", ""))
                if len(curr) > 0:
                    values.append(curr)
                    curr = []
            elif len(parts) == 2:
                curr.append(parts[1])
if len(curr) > 0:
    values.append(curr)
    curr = []

d = {}



In [45]:
len(values[75]), seccs[75]

(202, 'CachedCompile')

In [87]:
rows = []
for idx in range(len(values[0])):
    d = {}
    for idxcol, col in enumerate(seccs):
        if col in ["CreateXlaTensor", "DestroyDataHandles", "DestroyXlaTensor",  "ReleaseDataHandles", "CreateDataHandles"]: continue
        try:
            d[col] = values[idxcol][idx]
        except:
            d[col] = -999
    rows.append(d)

In [88]:
df = pd.DataFrame(rows, columns=seccs, index=pd.RangeIndex(len(values[0]), name='x'))

In [89]:
df

,CompileTime,DeviceLockWait,ExecuteTime,InboundData,InputOutputAliasCount,IrValueTensorToXlaData,OutboundData,ReleaseDataHandlesTime,TensorsGraphSize,TransferFromServerTime,...,xla::unsqueeze,xla::view,xla::zero_,XrtAllocateFromTensor,XrtCompile,XrtExecute,XrtExecutorEvict,XrtReadLiteral,XrtReleaseAllocation,CachedCompile
x,,,,,,,,,,,,,,,,,,,,,
0,3.0,3.0,2.0,2.0,1.0,118.0,130.0,3.0,3.0,2.0,...,1.0,9.0,92.0,7656.0,18.0,2063.0,0.0,1033.0,14957.0,4.0
1,7.0,7.0,6.0,5.0,2.0,122.0,140.0,11.0,7.0,5.0,...,2.0,18.0,138.0,7666.0,22.0,2067.0,0.0,1037.0,14965.0,8.0
2,7.0,11.0,10.0,8.0,2.0,126.0,150.0,25.0,11.0,8.0,...,3.0,27.0,184.0,7676.0,22.0,2071.0,0.0,1041.0,14979.0,12.0
3,7.0,15.0,14.0,11.0,2.0,130.0,160.0,31.0,15.0,11.0,...,4.0,36.0,230.0,7686.0,22.0,2075.0,0.0,1045.0,14985.0,16.0
4,7.0,19.0,18.0,14.0,2.0,134.0,170.0,39.0,19.0,14.0,...,5.0,45.0,276.0,7696.0,22.0,2079.0,0.0,1049.0,14993.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,7.0,799.0,798.0,599.0,2.0,914.0,2120.0,3450.0,799.0,599.0,...,200.0,1800.0,9246.0,9646.0,22.0,2859.0,0.0,1829.0,18404.0,800.0
200,7.0,803.0,802.0,602.0,2.0,918.0,2130.0,3465.0,803.0,602.0,...,201.0,1809.0,9292.0,9656.0,22.0,2863.0,0.0,1833.0,18419.0,804.0
201,7.0,807.0,806.0,605.0,2.0,922.0,2140.0,3477.0,807.0,605.0,...,202.0,1818.0,9338.0,9666.0,22.0,2867.0,0.0,1837.0,18431.0,808.0


In [90]:
import altair as alt
import pandas as pd
import numpy as np

alt.data_transformers.disable_max_rows()

#del(df["CreateXlaTensor"])
#del(df["DestroyDataHandles"])
#del(df["DestroyXlaTensor"])


source = df #pd.DataFrame(np.cumsum(np.random.randn(100, 3), 0).round(2), index=pd.RangeIndex(100, name='x'))
source = source.reset_index().melt('x', var_name='category',value_name='y')

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='x:Q',
    y='y:Q',
    color='category:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='x:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'y:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='x:Q',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

alt.LayerChart(...)